In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import pandas as pd

In [3]:
# selenium

chat_list=[] ; date_list = [] ; view_list = [] ; comments_list = []
chatlst=[] ; datelst = [] ; viewlst = [] ; commentslst = [] 

for i in range (1, 6):
    URL='https://comic.naver.com/bestChallenge/list.nhn?titleId=739130&page={}'.format(i)
    driver=webdriver.Chrome('chromedriver.exe')
    driver.get(URL)

    time.sleep(1)
    
    chat_list.clear()

    #이동한 페이지 주소 읽고, 파싱하기
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser') 

    length=driver.find_elements_by_class_name('title')

    webnum=[y.text for y in length]

    enterToon = driver.find_elements_by_partial_link_text(webnum[0])
    
    time.sleep(0.5)
    
    enterToon[0].click()

    # time.sleep(3)
    
    person= driver.find_elements_by_class_name('date')
    view_list.append(person[1].text)

    time.sleep(0.5)

    try:
        html = driver.current_url
        currentPage=html[html.find('?'):]
    except:
        continue
        
    #댓글창 페이지 URL
    URL2='https://comic.naver.com/comment/comment.nhn' + currentPage
        
    time.sleep(0.4)
        
    #댓글창 페이지 열기
    driver2 = webdriver.Chrome('chromedriver.exe')
    driver2.get(URL2)
        
    time.sleep(1)
    
    comments = driver2.find_elements_by_class_name('u_cbox_count')
    comments_list.append(comments[0].text)

    time.sleep(0.5)

    # chatbest = driver2.find_elements_by_class_name('u_cbox_contents')

    time.sleep(1)

    # jj = 0

    # while jj<len(chatbest):
    #     chat_list.append(chatbest[jj].text)
    #     jj +=1

    time.sleep(0.3)
            
    #댓글 페이지 닫기
    error_back=0
    driver2.close()
            
    time.sleep(0.4)
    
    #페이지 뒤로 가기, 다시 만화 목록으로 
    driver.back()
    
    # chatbest.clear()

    #전체 베스트도전 만화 창으로 이동
    time.sleep(0.5)
    # chatlst.append(chat_list.copy())
    commentslst.append(comments_list.copy())
    viewlst.append(view_list.copy())

    driver.close()
    
    time.sleep(0.5)

    comments_list.clear()
    view_list.clear()
    length.clear()

    #---------------------------------------------------크롤링 END--------------------------------------------------------------

#전체 데이터를 데이터 프레임으로 변환
df = pd.DataFrame()
df['조회수'] = viewlst
df['댓글수'] = commentslst
# df['댓글'] = chatlst
df.to_csv('대학원탈출일지.csv', index=False, encoding='utf-8-sig')

In [14]:
dfdf = pd.read_csv('베스트도전/NaverWebtoon.csv', encoding='cp949')

In [17]:
title = []
for i in range(0,144):
    title.append(dfdf['title'][i])

In [29]:
df = pd.DataFrame()

In [30]:
# selenium으로 베스트 도전 1 ~ 6페이지 불러오기

URL='https://comic.naver.com/genre/bestChallenge.nhn?m=main&order=ViewCount&page=1'
driver=webdriver.Chrome('chromedriver.exe')
driver.get(URL)

time.sleep(1)

chat_list=[] ; date_list = [] ; view_list = [] ; comments_list = []
chatlst=[] ; datelst = [] ; viewlst = [] ; commentslst = [] 

for i in range(0, 24):
    
    time.sleep(1)
    
    #전체 웹툰 목록 중 월요일 첫 번째 웹툰으로 페이지 이동
    page=driver.find_elements_by_class_name('challengeTitle')
    page[i].click()
    
    time.sleep(0.5)
    titles = title[i]
    #이동한 페이지 주소 읽고, 파싱하기
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    length=driver.find_elements_by_class_name('title')
    if(len(length)>5):
        length=length[:10]
        #각 회차를 돌며 댓글 작성자 아이디 수집
    for j in range(len(length)):
        #해당 페이지의 회차 모두 가져오기
        titlenum=driver.find_elements_by_class_name('title')
        
        time.sleep(0.5)
        
        webnum=[y.text for y in titlenum]
        enterToon = driver.find_elements_by_partial_link_text(webnum[j])
        
        time.sleep(0.5)
        
        enterToon[0].click()

        # 조회수, 연재일 수집
        person= driver.find_elements_by_class_name('date')
        view_list.append(person[1].text)
        date_list.append(person[0].text)
        if j == 0:
            df['{} 조회수'.format(titles)] = view_list
        else:
            df.append({'{} 조회수'.format(titles): view_list}, ignore_index=True)
        
        if j == 0:
            df['{} 연재일'.format(titles)] = date_list
        else:
            df.append({'{} 연재일'.format(titles): date_list}, ignore_index=True)
        #현재 댓글 부분 파싱이 안되는 설정이므로 src의 댓글창 페이지로 직접 이동
        #현재 URL주소 복사
        try:
            html = driver.current_url
            currentPage=html[html.find('?'):]
        except:
            continue
        
        #댓글창 페이지 URL
        URL2='https://comic.naver.com/comment/comment.nhn' + currentPage
        
        time.sleep(0.4)
        
        #댓글창 페이지 열기
        driver2 = webdriver.Chrome('chromedriver.exe')
        driver2.get(URL2)
        
        time.sleep(2)
        
        
        #댓글 수 수집
        comments = driver2.find_elements_by_class_name('u_cbox_count')
        comments_list.append(comments[0].text)

        if j == 0:
            df['{} 댓글 수'.format(titles)] = comments_list
        else:
            df.append({'{} 댓글 수'.format(titles): comments_list}, ignore_index=True)

        time.sleep(0.5)

        #댓글 페이지 닫기
        driver2.close()
                
        time.sleep(0.4)
        
        #페이지 뒤로 가기, 다시 만화 목록으로 
        driver.back()
        
        time.sleep(1)
        
        #time.sleep(0.5)
        
        enterToon.clear()
        
        #time.sleep(0.5)
        comments_list.clear()
        view_list.clear()
        date_list.clear()
    
    #전체 베스트도전 만화 창으로 이동
    time.sleep(1)

    driver.back()

    time.sleep(0.5)

    comments_list.clear()
    view_list.clear()
    length.clear()

#---------------------------------------------------크롤링 END--------------------------------------------------------------

#데이터 프레임을 CSV파일로 추출 내보내기(ID, CHAT 따로)
# df = pd.DataFrame()
# df['조회수'] = viewlst
# df['댓글수'] = commentslst
# # df['댓글'] = chatlst
# df.to_csv('대학원탈출일지.csv', index=False, encoding='utf-8-sig')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=90.0.4430.93)


In [11]:
df = pd.DataFrame()
df['연재날짜, 조회수'] = viewlst
df['댓글수'] = commentslst
# df['댓글'] = chatlst
df.to_csv('여우자매.csv', index=False, encoding='utf-8-sig')